# 空間を扱うプログラミング

## §1 座標系 (coordinate system)

このセクションでは、3Dグラフィックスで用いられる主な座標系について学びます。ここでは基本的に直交座標系を前提としています。これは、座標軸が互いに直角を成すシステムです。

### 主な座標系

* ワールド座標系
  * 3D空間全体に対する絶対座標を表す座標系で、座標軸は x, y, z と小文字で表記します。
* オブジェクトのローカル座標系
  * 各オブジェクト固有の相対座標を表し、オブジェクトの動きによってその位置や向きが変わります。
  * 視点がオブジェクト自身である場合、この座標系は固定されているように見えます。
  * 座標軸は X, Y, Z と大文字で表記します。
* カメラ座標系
  * カメラ自体の位置と向きを基準にした座標系で、カメラがどの方向を向いているかを示します。
  * 座標軸はカメラの視線方向、縦方向、横方向を定義し、X, Y, Z で表されます。
* 画像座標系
  * カメラによって捉えられた画像上での座標を定義する座標系で、左上隅を原点とし、右方向を +X、下方向を +Y とします。


### 右手系 (right-handed system)・左手系 (left-handed system)

* 3D座標系では、座標軸の向きに右手系と左手系の二つの方法があります。
* 右手を使って親指、人差し指、中指がそれぞれ +x, +y, +z を示すようにし、左手でも同様にしてみると、両手系が重ならないことが理解できます。

<img alt="Right-handed and left-handed coordinate systems" src="./notebook_assets/images/1_right-left-handed-corrdinate-system.png" style="width:600px;" />


### 使用ツールとライブラリ

* Open3D: この研修では、Open3Dライブラリを用いて3Dオブジェクトの基本的な表示や演算を行います。演習の主目的はオブジェクトの位置や姿勢の表現を理解することです。

### 演習環境の設定

* 演習では以下のような環境を使用します。
  * ワールド座標軸は赤、緑、青の矢印で示され、それぞれ +x, +y, +z 方向を表しています。
    * 矢印の根本はワールド座標原点から少しずれていることに注意してください。
  * ワールドには床を表す平面があり、xz 面に平行です
  * また、カメラオブジェクトはワールドの原点に位置し、レンズの向きが +y 方向になっています。

   <img alt="Initial world setting" src="./notebook_assets/images/1_initial_world.png" style="width:600px;"/>


In [ ]:
# 必要なライブラリをインポート
from __future__ import annotations

import numpy as np
import open3d as o3d
import sympy as sp
from IPython.display import Math, display
from scipy.spatial.transform import Rotation, Slerp

from util_lib.transformable_object import TransformableObject
from util_lib.types import EulerOrder, Transform
from util_lib.visualization import draw_geometries
from util_lib.world import create_coordinate_objects

# ワールド・カメラ座標系の作成と読み込み
world_coordinate: o3d.geometry.Geometry = create_coordinate_objects()
original_camera = TransformableObject.load_model("./data/camera.gltf", enable_post_processing=True)

# 行列表示用の関数
def print_matrix(mat: np.ndarray | Transform) -> None:
    if isinstance(mat, Transform):
        mat_sym = sp.Matrix(mat.get_matrix())
    else:
        mat_sym = sp.Matrix(mat)
    mat_sym = mat_sym.applyfunc(lambda x: sp.Symbol(f"{x:.3f}"))
    display(Math(sp.latex(mat_sym)))


#### [課題1] ワールドの確認

* (1-1) プリセットとして用意されたワールドを表示しよう
* (1-2) ワールドの広がりに対するワールド座標軸、カメラのローカル座標軸の向きを確認しよう
  * ワールド座標軸の向きの関係を見て、右手系であることを確認してください
  * グレーの平面を水平な地面とみなしたとき、ワールド座標軸が向いている方向を確認してください
    * 上下・左右・前後と座標軸の向きの関係を見てください
* 確認ができたら次のステップに進むため、Open3D ビューワーを閉じてください (ビュ-ワーの❌️ボタンを押すか、ビュワーウィンドウがアクティブな状態で `ESC` キーを押す)

(参考) Open3D ビューワー操作
* 左クリックでドラッグ: 注視点まわりにカメラを回転
* スクロール: 中心点にカメラを近づける or 遠ざける
* Ctrl+左ドラッグ: 注視点を移動
* Shift+左ドラッグ: 視線方向に対しカメラを回転

In [ ]:
# ワールド座標系とカメラの初期位値を表示
#
# 下記のような警告は問題ないので無視してください。
# [Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display

draw_geometries(world_coordinate, original_camera)

## §2 ワールド座標に対するオブジェクトの操作

* オブジェクトに対して定義される操作には下記があります。
  * 並進 (translate, translation)
  * 回転 (rotate, rotation)
  * 拡大縮小 (scale, scaling)
    * 鏡映、反転を含む (負の拡大率を定義する)
* これらの操作をまとめて transform、transformation と呼びます。

オブジェクトの回転・並進の例

* ワールドの +y 方向にあるカメラが回転・並進変換後のオブジェクト
  1. ワールド x 軸周りに90度回転
  2. ワールド y 軸方向に+1並進

<img alt="Example of object transformation" src="./notebook_assets/images/1_example_transformation.png" style="width:600px;"/>

In [ ]:
# Just example

# オブジェクトを新規にコピー
camera = original_camera.copy()

# コピーしたオブジェクトを変換
camera.translate((0, 1, 0)) # y軸方向に1m移動
camera.rotate_by_euler(EulerOrder.xyz, (90, 0, 0)) # x軸周りに90度回転

draw_geometries(world_coordinate, original_camera, camera)

## §2.1 オブジェクトの並進

* 並進とは、オブジェクトをある方向に移動させる操作です。これは3次元の並進ベクトル (dx, dy, dz) によって表現されます。
  * オブジェクトの現在位置にこのベクトルを加えることで、新しい位置が決まります。
  * 繰り返し並進を行うと、オブジェクトは次第に位置を変えていきます。

#### [課題2] オブジェクトの並進を試してみよう

* (2-1) カメラオブジェクトを y 方向に +1 単位移動させてください。
* (2-2) カメラオブジェクトをまず x 方向に +1 単位、次に z 方向に -1 単位移動させてください。
* (2-3) 一度の操作でカメラオブジェクトを(2-2)の最終位置に移動させてください。これは並進ベクトルの合成を意味します。

In [ ]:
# (2-1)
camera = original_camera.copy()
camera.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera)

In [ ]:
# (2-2)
camera_2_2 = original_camera.copy()
camera_2_2.translate() # implement here
camera_2_2.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera_2_2)

In [ ]:
# (2-3)
camera_2_3 = original_camera.copy()
camera_2_3.translate() # implement here
draw_geometries(world_coordinate, original_camera, camera_2_2, camera_2_3)

## §2.2 オブジェクトの姿勢回転

3D空間でのオブジェクトの姿勢を変更するには、回転操作が必要です。以下は回転を表現する主な方法です：

* Euler (オイラー) 角
* 回転行列
* クォータニオン
* 回転ベクトル (ここでは扱いません)

### §2.2.1 Euler 角

Euler 角では、オブジェクトの姿勢を座標軸ごとの回転の組み合わせで表します。

* 角度の単位
  * 通常、度数法（度）が使われますが、科学的な文脈ではラジアンが用いられることもあります。
* 回転の方向
  * 右手系で考える場合、「右ネジの法則」に従います。つまり、右手の親指を座標軸の正の方向に向け、他の指が握る方向が回転の正の方向です。

#### Euler 角の例

考えてみましょう：オブジェクトがまず x 軸周りに 40° 回転、続いて y 軸周りに 30° 回転、最後に z 軸周りに 30° 回転します。この場合、Euler 角は (rx, ry, rz) = (40, 30, 30) として表され、回転順序は x → y → z です。

<img alt="Synthesis of Euler angles" src="./notebook_assets/images/1_synthesis_of_euler_rotation.png" style="width:400px;" />


In [ ]:
# 3軸の回転を一度に与える (回転順序はワールド座標軸の x → y → z)
camera_final = original_camera.copy()
camera_final.translate((1, 1.5, 0))
camera_final.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# x → y → zと1軸ずつ回転する
camera_x_rotated = original_camera.copy()
camera_x_rotated.translate((0, 0.5, 0))
camera_x_rotated.rotate_by_euler(EulerOrder.xyz, (40, 0, 0))

camera_xy_rotated = camera_x_rotated.copy()
camera_xy_rotated.translate((0, 0.5, 0))
camera_xy_rotated.rotate_by_euler(EulerOrder.xyz, (0, 30, 0))

camera_xyz_rotated = camera_xy_rotated.copy()
camera_xyz_rotated.translate((0, 0.5, 0))
camera_xyz_rotated.rotate_by_euler(EulerOrder.xyz, (0, 0, 30))

# 最終的な姿勢は同じ
draw_geometries(world_coordinate, original_camera, camera_final,
                camera_x_rotated, camera_xy_rotated, camera_xyz_rotated)

### 回転操作の非可換性

回転の順序を変えると、オブジェクトの最終的な姿勢も変わることに注意してください。

例えば、x → y → z の順に回転させるのと、z → y → x の順に回転させるのでは、結果が異なります。


<img alt="Non-commutativity of rotations" src="./notebook_assets/images/1_non_commutativity_of_rotation.png" style="width:600px;" />

In [ ]:
# 3軸の回転を与える (回転順序はワールド座標軸の x → y → z)
camera_rotate_xyz = original_camera.copy()
camera_rotate_xyz.translate((-0.5, 1, 0))
camera_rotate_xyz.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# 3軸の回転を与える (回転順序はワールド座標軸の z → y → x)
camera_rotate_zyx = original_camera.copy()
camera_rotate_zyx.translate((0.5, 1, 0))
camera_rotate_zyx.rotate_by_euler(EulerOrder.zyx, (30, 30, 40))

draw_geometries(world_coordinate, original_camera, camera_rotate_xyz, camera_rotate_zyx)

#### [課題3] オブジェクトをオイラー角で回転してみよう

* (3-1) カメラをワールド座標軸の z → y → x の順にそれぞれ 45°、90°、-45°回転してみましょう。
  * まず、どのような結果が得られるか頭の中で考えてください。
  * 想定した結果とあっているか確認してください。

In [ ]:
# (3-1)
camera = original_camera.copy()
camera.translate((-0.5, 1.5, 0))
camera.rotate_by_euler() # implement here

# 1軸ずつ回転していく様子を確認しよう
camera_z = original_camera.copy()
camera_z.translate((0.5, 0.5, 0))
camera_z.rotate_by_euler() # implement here

camera_zy = camera_z.copy()
camera_zy.translate((0, 0.5, 0))
camera_zy.rotate_by_euler() # implement here

camera_zyx = camera_zy.copy()
camera_zyx.translate((0, 0.5, 0))
camera_zyx.rotate_by_euler() # implement here

draw_geometries(world_coordinate, original_camera, camera, camera_z, camera_zy, camera_zyx)

### (参考) 外因性 (extrinsic) 回転と内因性 (intrinsic) 回転

#### 外因性 (Extrinsic) 回転

外因性回転とは、ワールド座標軸を基準にした回転です。これはオブジェクトが空間内でどのように位置変更をするかを表します。

回転順序を小文字の `xyz` などで表し、ワールド座標軸に対するオブジェクトの方向変更を指します。


#### 内因性 (Intrinsic) 回転

内因性回転は、オブジェクト自身のローカル座標軸に対する回転を意味します。オブジェクトが回転すると、その後の回転軸も一緒に回転するため、観察者から見るとオブジェクトの向きが変わっていくのが分かります。

内因性回転の回転順序は大文字の `XYZ` などで表されます。

#### 回転の違い

たとえ角度と回転の順序が同じであっても、外因性回転と内因性回転では最終的なオブジェクトの姿勢が異なります。

この違いは、回転がどの座標系を基準に行われているかによって決まります。

外因性回転ではワールド座標系が常に基準となり、内因性回転ではオブジェクトのローカル座標系が基準となります。


<img alt="Extrinsic and intrinsic rotation" src="./notebook_assets/images/1_extrinsic_and_intrinsic_rotation.png" style="width:600px;" />

### §2.2.2 回転行列

#### 回転行列の基本

オブジェクトの回転を数学的に表現する一つの方法は、回転行列を使用することです。

3次元空間内の任意の回転は、3x3の行列で表現可能です。

このような行列を特に「回転行列」と呼びます。


#### 回転行列の特徴

* 明確な数学的表現：Euler 角のように順序や外因性・内因性の違いに悩まされることなく、回転を明確に表現できます。
* 単軸回転と多軸回転：回転行列では、一つの軸周りの回転だけでなく、複数の軸周りの組み合わせた回転も表現できます。
* 合成が簡単：回転の合成が行列の積として直接表現でき、計算が容易です。
* 統一的な扱い: 回転行列は、並進変換と組み合わせることで、座標変換を統一的に扱うことが可能です。これにより、3D空間内でのオブジェクトの位置と姿勢の変更を一元的に表現できます。
* 冗長性とリーダビリティ：回転を表すために必要なパラメータが9つあり、Euler 角よりもパラメータ数が多いため、視覚的に理解しにくい面もあります。


#### 回転の合成

複数の回転を組み合わせる場合、それぞれの回転行列の積で表されます。

たとえば、回転行列 $\mathbf{R}_1$、$\mathbf{R}_2$、$\mathbf{R}_3$ がある場合、これらを組み合わせる操作は行列の積 $\mathbf{R}_3 \mathbf{R}_2 \mathbf{R}_1 \mathbf{x}$ として計算されます。ここで $\mathbf{x}$ はオブジェクトの座標集合です。


#### 回転行列の生成

Euler 角から回転行列を生成する方法があります。

Python で扱う場合は、[`scipy.spatial.transform.Rotation`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) モジュールが便利です。

例えば、ワールド座標系において x、y、z 軸を順に 40°、30°、30° 回転させる回転行列を下記のように生成することができます。

```python
from scipy.spatial.transformation import Rotation

rotation = Rotation.from_euler("xyz", (40, 30, 30), degrees=True)
rotation_mat = rotation.as_matrix()
```


In [ ]:
# 回転しない場合の回転行列は単位行列に等しい
identity_matrix = Rotation.from_euler(EulerOrder.xyz, (0, 0, 0), degrees=True).as_matrix()
print("Identity matrix")
print_matrix(identity_matrix)

# x軸周りに40度、y軸周りに30度、z軸周りに30度回転する場合の回転行列
rotate_matrix = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_matrix()
print("3軸回転行列")
print_matrix(rotate_matrix)

# 1軸ずつ回転する場合の回転行列
rotate_matrix_x = Rotation.from_euler(EulerOrder.xyz, (40, 0, 0), degrees=True).as_matrix()
rotate_matrix_y = Rotation.from_euler(EulerOrder.xyz, (0, 30, 0), degrees=True).as_matrix()
rotate_matrix_z = Rotation.from_euler(EulerOrder.xyz, (0, 0, 30), degrees=True).as_matrix()
print("1軸ずつ回転した場合の回転行列")
print_matrix(rotate_matrix_x)
print_matrix(rotate_matrix_y)
print_matrix(rotate_matrix_z)

# 1軸ずつ回転した場合の回転行列をまとめる
rotate_matrix_xyz = rotate_matrix_z @ rotate_matrix_y @ rotate_matrix_x
print("1軸ずつの回転を合成した回転行列")
print_matrix(rotate_matrix_xyz)

# 2通りで計算した行列が等しいことの確認 (等しくないと例外が発生する)
np.testing.assert_array_almost_equal(rotate_matrix, rotate_matrix_xyz)

In [ ]:
# オイラー角による回転
camera_by_euler = original_camera.copy()
camera_by_euler.translate((-0.5, 1, 0))
camera_by_euler.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# 回転行列による回転
rotate_matrix = [
    [0.750, -0.105, 0.653],
    [0.433, 0.824, -0.365],
    [-0.500, 0.557, 0.663],
]

camera_by_matrix = original_camera.copy()
camera_by_matrix.translate((0.5, 1, 0))
camera_by_matrix.rotate(rotate_matrix)

draw_geometries(world_coordinate, original_camera, camera_by_euler, camera_by_matrix)

### §2.2.3 クォータニオン (Quaternion)

#### Euler 角と回転行列の課題

Euler 角と回転行列は多くの3Dアプリケーションで使われますが、いくつかの課題があります。
特に、回転行列はパラメータが多く冗長ですし、Euler 角と回転行列の両方とも、異なる姿勢間の滑らかな遷移（補間）を表現するのが難しいです。

#### クォータニオンの利点

* クォータニオンは、回転を4つのパラメータ（qw, qx, qy, qz）で表現する方法です。これにより、回転行列よりもパラメータが少なく、より効率的に回転を扱うことができます。
* クォータニオンは、回転の順序や外因性・内因性のような曖昧さを排除し、一貫性のある回転を保証します。
* 特に、異なる姿勢間の補間が非常に簡単で、滑らかなアニメーションやリアルタイムの3D回転に適しています。

#### クォータニオンの計算

* Pythonでクォータニオンを扱う際には、[`scipy.spatial.transform.Rotation`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) モジュールが便利です。このモジュールを使用することで、クォータニオンを簡単に生成、操作、変換することができます。
  * scipy パッケージ ではクォータニオンの要素の順番が (qx, qy, qz, qw) と定義されていますので、これを意識して利用する必要があります。

#### クォータニオンの応用

* クォータニオンは、ゲーム開発、ロボティクス、航空宇宙など、多岐にわたる分野で重要な役割を果たします。これにより、3D空間内でのオブジェクトの動きを自然かつ効率的にシミュレートすることが可能になります。


In [ ]:
# 回転しない場合のクォータニオンは qw だけ1で他が0
identity_quaternion = np.array([0, 0, 0, 1]) # (qx, qy, qz, qw)
np.testing.assert_array_almost_equal(
    identity_quaternion,
    Rotation.from_euler(EulerOrder.xyz, (0, 0, 0), degrees=True).as_quat(),
)

# x軸周りに40度、y軸周りに30度、z軸周りに30度回転する場合のクォータニオン
rotate_quaternion = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_quat()
print_matrix(rotate_quaternion)

In [ ]:
# Euler 角による回転
camera_by_euler = original_camera.copy()
camera_by_euler.translate((-0.5, 1, 0))
camera_by_euler.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

# クォータニオンによる回転
rotate_quaternion_xyzw = np.array([0.256, 0.320, 0.149, 0.900])
camera_by_quaternion = original_camera.copy()
camera_by_quaternion.translate((0.5, 1, 0))
camera_by_quaternion.rotate_by_quaternion(rotate_quaternion_xyzw)

draw_geometries(world_coordinate, original_camera, camera_by_euler, camera_by_quaternion)

### (参考) 回転の補間

#### Euler 角による補間

この例では、Euler 角の回転順序 xyz で、初期姿勢 (0, 0, 0) から目標姿勢 (40°, 30°, 30°) への補間を行います。これは、開始角度から終了角度までの中間の角度を計算する過程です。

#### クォータニオンを使った補間

クォータニオンを用いた補間は、主に2種類の方法があります：線形補間と球面線形補間（SLERP）。

1. 線形補間:
    * 二つのクォータニオン $\mathbf{q}_1$ と $\mathbf{q}_2$ の間で、パラメータ $r$（0から1の間の値）を使って線形補間を行います。
    * 補間されたクォータニオンは $\mathbf{q}(r) = (1-r) \mathbf{q}_1 + r \mathbf{q}_2$ と計算され、これはクォータニオンをベクトルとして扱い、その加重平均を取ることに相当します。
2. 球面線形補間 (SLERP):
    * SLERPは、クォータニオン間の最短経路を考慮した補間で、より自然な回転変化を提供します。この方法は、scipy パッケージの [`scipy.spatial.transform.Slerp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Slerp.html) を使用して簡単に実装できます。

補間の視覚化

下図には、線形補間と球面線形補間の結果を示しています。左側が線形補間による中間姿勢、真ん中が目標姿勢、右側が球面線形補間による中間姿勢です。この比較から、SLERPがどのように滑らかな回転を提供するかが視覚的に理解できます。


<img alt="Interpolating rotations" src="./notebook_assets/images/1_interpolating_rotation.png" style="width:600px;" />

In [ ]:
class LinearInterpolation:

    """scipy.spatial.transform.Slerp と同じインターフェースを持つ線形補間クラス."""

    def __init__(self, key_times, key_values):
        self.__key_times = key_times
        self.__key_values = key_values

    def __call__(self, time):
        return self.__interpolate(time)

    def __interpolate(self, time):
        if time <= self.__key_times[0]:
            return self.__key_values[0]
        if time >= self.__key_times[-1]:
            return self.__key_values[-1]

        for i in range(len(self.__key_times) - 1):
            if self.__key_times[i] <= time < self.__key_times[i + 1]:
                t = (time - self.__key_times[i]) / (self.__key_times[i + 1] - self.__key_times[i])
                return self.__key_values[i] * (1 - t) + self.__key_values[i + 1] * t

        raise ValueError("Invalid time")

In [ ]:
key_times = [0, 1]
key_euler_rotations = [(0, 0, 0), (40, 30, 30)]
rotations = Rotation.from_euler(EulerOrder.xyz, key_euler_rotations, degrees=True)

# 補間クラスのインスタンス生成
linear = LinearInterpolation(key_times, [r.as_quat() for r in rotations])
slerp = Slerp(key_times, rotations)


final_camera = original_camera.copy()
final_camera.translate((0, 1.5, 0))
final_camera.rotate_by_euler(EulerOrder.xyz, key_euler_rotations[1])

print("線形補間")
cameras_linear = []
for time in np.arange(0, 1.1, 0.1):
    interpolated_quat = linear(time)
    print(f"Time: {time:.1f}, Quaternion: {interpolated_quat}")

    camera_by_linear = original_camera.copy()
    y_pos = time * 1.5
    camera_by_linear.translate((-1, y_pos, 0))
    camera_by_linear.rotate_by_quaternion(interpolated_quat)
    cameras_linear.append(camera_by_linear)

print("球面線形補間")
cameras_slerp = []
for time in np.arange(0, 1.1, 0.1):
    interpolated_quat = slerp(time).as_quat()
    print(f"Time: {time:.1f}, Quaternion: {interpolated_quat}")

    camera_by_slerp = original_camera.copy()
    y_pos = time * 1.5
    camera_by_slerp.translate((1, y_pos, 0))
    camera_by_slerp.rotate_by_quaternion(interpolated_quat)
    cameras_slerp.append(camera_by_slerp)
draw_geometries(world_coordinate, *cameras_linear, *cameras_slerp, final_camera)

## §2.3 トランスフォーメーション (Transformation)

### 並進と回転の統合

前のセクションで、3次元空間でのオブジェクトの並進と回転を別々に扱いました。ここでは、これらを一つの行列で統一的に扱う方法、トランスフォーメーションについて説明します。

#### トランスフォーメーション行列

トランスフォーメーションは、4x4の行列を使用して表されます。この行列は、並進と回転を同時に表現することができます。

* 並進ベクトル：並進を表すベクトルは $[t_x, t_y, t_z]$ と表されます。
* 回転行列：回転を表す3x3部分行列 $\mathbf{R}$ は、その成分を $r_{11}$ から $r_{33}$ までで表します。
* トランスフォーメーション行列：

$$
\mathbf{T} = \begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

#### トランスフォーメーション行列の使用

* 並進と回転の同時適用：この行列をオブジェクトの座標ベクトルに適用することで、並進と回転を一度に行うことができます。これは3D空間におけるオブジェクトの位置と姿勢を表すのに非常に便利です。
* 複数のトランスフォーメーションの組合せ：複数のトランスフォーメーションを組み合わせる場合は、それぞれのトランスフォーメーション行列を積算することで表現されます。例えば、$\mathbf{T}_1$、$\mathbf{T}_2$、$\mathbf{T}_3$ の順に適用する場合は、$\mathbf{T}_3 \mathbf{T}_2 \mathbf{T}_1 \mathbf{x}$ となります。
* 逆トランスフォーメーション：トランスフォーメーション行列の逆行列を計算し適用することで、オブジェクトを元の位置や姿勢に戻すことができます。具体的には $\mathbf{T}^{-1} \mathbf{T} \mathbf{x} = \mathbf{I} \mathbf{x} = \mathbf{x}$ となり、元の座標 $\mathbf{x}$ を復元できます。



In [ ]:
camera = original_camera.copy()
camera.translate((-1, 1.5, 1))
camera.rotate_by_euler(EulerOrder.xyz, (40, 30, 30))

camera_transformed = original_camera.copy()
rotation_matrix = Rotation.from_euler(EulerOrder.xyz, (40, 30, 30), degrees=True).as_matrix()

print("回転行列")
print_matrix(rotation_matrix)
transformation = Transform([
    [ 0.750, -0.105,  0.653,  -1],
    [ 0.433,  0.824, -0.365, 1.5],
    [-0.500,  0.557,  0.663,   1],
    [     0,      0,      0,   1],
])

camera_transformed.transform(transformation)

draw_geometries(world_coordinate, original_camera, camera, camera_transformed)

In [ ]:
# 逆トランスフォーメーション

transformation = Transform([
    [ 0.750, -0.105,  0.653,  -1],
    [ 0.433,  0.824, -0.365, 1.5],
    [-0.500,  0.557,  0.663,   1],
    [     0,      0,      0,   1],
])

inverse_transformation = transformation.inv()
np.testing.assert_array_almost_equal((inverse_transformation @ transformation).get_matrix(), np.identity(4))

camera_transformed = original_camera.copy()
camera_transformed.transform(transformation)
camera_transformed.transform(inverse_transformation)

# original_camera, camera_transformed はピッタリ重なる
draw_geometries(world_coordinate, original_camera, camera_transformed)

### 環境による座標系の違い

異なるプラットフォームやツールを使用する際、それぞれの座標系の違いに注意が必要です。

各プラットフォームやツールは独自の座標軸の向きや長さの単位を持つことがあります。

これらの違いを適切に管理しないと、オブジェクトの位置や姿勢が期待したものと異なる結果になることがあります。

#### 例: Open3D と Unity での座標系の違い

ここでは、3DオブジェクトをOpen3DとUnityでどのように異なる座標系で表示されるかを比較します。

1. Unityでのプロジェクト作成:
    * Unity Hubを開き、「New project」で新しいプロジェクトを作成します。
2. オブジェクトのインポート:
    * Unity エディタの Assets 部分にある data/camera_for_unity/ ディレクトリから、camera.gltf ファイルをドラッグアンドドロップしてインポートします。

<img alt="Import GLTF object into Unity project" src="./notebook_assets/images/1_import_gltf_to_unity.png" style="width:800px;" />

3. オブジェクトの配置:
    * インポートされた camera_for_unity の中から camera.gltf をシーン内にドラッグアンドドロップして配置します。

<img alt="Locate GLTF object into Unity project" src="./notebook_assets/images/1_locate_gltf_in_unity_space.png" style="width:800px;" />

この手順により、Unity内で3Dオブジェクトがどのように表示されるかを視覚的に確認できます。この比較から、Open3DとUnityの座標系の違いを理解することが可能になります。

#### [課題4] Open3D と Unity でワールド座標系がどのように違うか確認しよう

* (4-1) Unity は右手系でしょうか、左手系でしょうか?
* (4-2) Open3D と Unity でワールド座標系の向きはどのように違うでしょうか?
* (4-3) Open3D と Unity で、オブジェクトを同じ方向に動かすためにどのような操作が必要か考えてみましょう
  * Open3D のワールド座標系に対し、カメラオブジェクトを (x, y, z) = (1, 1, 1) 移動させます
  * Unity 上でこれと同じ方向にカメラオブジェクトを動かすために必要な並進 (x, y, z) はいくつになるでしょうか?